In [3]:
import numpy as np
import pandas as pd

In [4]:
df=pd.read_csv('BankFAQs.csv')
df.head()
# print(df.size)

,Question,Answer,Class
0,What are the documents required for opening a ...,Following documents are required to open a Cur...,accounts
1,Can I transfer my Current Account from one bra...,"Yes, Current Accounts can be transferred from ...",accounts
2,My present status is NRI. What extra documents...,NRI/PIO can open the proprietorship/partnershi...,accounts
3,What are the documents required for opening a ...,Following documents are required for opening a...,accounts
4,What documents are required to change the addr...,Following documents are required to change the...,accounts


In [5]:
print(df['Class'].unique())

['accounts' 'cards' 'fundstransfer' 'insurance' 'investments' 'loans'
 'security']


In [6]:
import pandas as pd

import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
import datetime

stop_words = set(stopwords.words('english'))
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [7]:
def cleanup(review):
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    lemmatizer = WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(all_stopwords)]
    print(type(review))
    return ' '.join(review) 

In [8]:
print(df['Question'][0])
sen=cleanup(df['Question'][0])
sen

What are the documents required for opening a Current Account of a sole proprietorship firm
<class 'list'>


'document required opening current account sole proprietorship firm'

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X=[]
m=0
y=[]
#preparing X and y to use
for i in range(len(df)):
  m=max(m,len(df['Question'][i]))
  X.append(cleanup(df['Question'][i]))
  y.append(df['Class'][i])

print(m)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [10]:
print(type(X))

print(len(X))
print(len(y))

<class 'list'>
1773
1773


In [11]:
#changing y from categorical to numerical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(y)
from collections import Counter
items = Counter(y).keys()
print(len(items))
y[0:5]

7


array([0, 0, 0, 0, 0], dtype=int64)

In [12]:
tfv = TfidfVectorizer(min_df=1, stop_words='english')
tfv.fit(X)

X = tfv.transform(X)

In [13]:
x=tfv.transform(['account are very important'])
x
print(x.shape)
print(type(x))

(1, 1052)
<class 'scipy.sparse.csr.csr_matrix'>


In [14]:
print(type(X))
print(type(y))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.20,random_state=42,shuffle=True)

In [16]:
print(X_train.shape)
print(y_train.shape)

(1418, 1052)
(1418,)


In [17]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)
class_=le.inverse_transform(model.predict(X))
print("SVC:", model.score(X_test, y_test))

SVC: 0.8929577464788733


In [18]:
from sklearn.ensemble import RandomForestClassifier
model1=RandomForestClassifier(n_estimators = 25, criterion = 'entropy', random_state = 2)
model1.fit(X_train, y_train)

print("RF:", model1.score(X_test, y_test))

RF: 0.9070422535211268


In [19]:
from sklearn.naive_bayes import GaussianNB
model2 = GaussianNB()
X_train=X_train.toarray()
X_test=X_test.toarray()
model2.fit(X_train, y_train)
print("Naive:", model2.score(X_test, y_test))

Naive: 0.8591549295774648


In [20]:
from sklearn.tree import DecisionTreeClassifier
model3 = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
model3.fit(X_train, y_train)

print("DT:", model3.score(X_test, y_test))


DT: 0.8873239436619719


In [21]:
ques='What are documents required to open an acount'

# ques=cleanup(ques[0])
ques_X=[]
ques_X.append(cleanup(ques))
print(ques)
# tfv.fit(ques_X)
ques_X = tfv.transform(ques_X)
ques_X
print(ques_X)

<class 'list'>
What are documents required to open an acount
  (0, 814)	0.5748147372950101
  (0, 639)	0.6014600562507082
  (0, 273)	0.5548277377018596


In [22]:
def get_max5(arr):
    ixarr = []
    for ix, el in enumerate(arr):
        ixarr.append((el, ix))
    ixarr.sort()
    ixs = []
    for i in ixarr[-5:]:
        ixs.append(i[1])
    return ixs[::-1]

def get_response(usrText):
    while True:
        if usrText.lower() == "bye":
            return "Bye"
        GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up", "hey","hiii","hii","yo"]
        a = [x.lower() for x in GREETING_INPUTS]
        sd=["Thanks","Welcome"]
        d = [x.lower() for x in sd]
        am=["OK"]
        c = [x.lower() for x in am]
       # ty = ["getting"]
       # r = [x.lower() for x in ty]
        t_usr = tfv.transform([cleanup(usrText.strip().lower())])
        class_ = le.inverse_transform(model.predict(t_usr))
        questionset = df[df['Class'].values == class_]
        cos_sims = []
        for question in questionset['Question']:
            sims = cosine_similarity(tfv.transform([question]), t_usr)

            cos_sims.append(sims)
        ind = cos_sims.index(max(cos_sims))
        b = [questionset.index[ind]]

        if usrText.lower() in a:

            return ("Hi, I'm Emily!\U0001F60A")

        if usrText.lower() in c:
            return "Ok...Alright!\U0001F64C"

        if usrText.lower() in d:
            return ("My pleasure! \U0001F607")

        if max(cos_sims) > [[0.]]:
            a = df['Answer'][questionset.index[ind]]+"   "
            return a
        elif max(cos_sims)==[[0.]]:
           return "sorry! \U0001F605"

In [23]:
get_response('document required for opening bank account')

<class 'list'>


'Following documents are required for opening Current Account of a company: Private Limited Companies - Memorandum of Association (MOA) - Articles of Association (AOA) - Certificate of Incorporation - Latest list of directors duly signed by any Director/Company Secretary /Authorised Signatory - Board Resolution (BR) duly signed by the directors of the company. Limited Companies - Memorandum of Association (MOA) - Articles of Association (AOA) - Certificate of Incorporation - Certificate of Commencement of Business - Latest list of directors duly signed by any Director/Company Secretary/Authorised Signatory - Board Resolution (BR) duly signed by the Directors/Company Secretary of the Company View more   '

In [24]:
get_response("loan policy")

<class 'list'>


'Premium Allocation Charge:This is a premium-based charge. After deducting this charge from your premiums, the remainder is invested to buy units. The remaining percentage of your premium is called the Premium Allocation Rate. The Premium Allocation Rate and Premium Allocation Charge are given in the table below. Single Premium Single Premium Top-Ups Premium Allocation Rate 97.50% 99% Premium Allocation Charge 2.50% 1% Fund Management Charge (FMC): The daily unit price already includes the fund management charge of 1.35% per annum charged daily, of the fundâ€™s value. Policy Administration Charge: The Policy Administration Charge is 0.13% per month of the total premiums paid (including any single premium top-ups).The charge will be deducted monthly, subject to a maximum charge of Rs. 500 per month. This charge will be taken by cancelling units proportionately from each of the fund(s) you have chosen. Mortality Charge:Every month a charge will be levied for providing you with the death 

In [25]:
get_response('document required to open bank account')

<class 'list'>


'To open a Savings Account you have to: Submit your identity proof and address proof along with your latest passport size photographs Submit self-attested copies and originals for verification View more   '

In [26]:
get_response('what should be the minimum balance in my acount')

<class 'list'>


'No minimum balance requirement.   '

In [27]:
pip install mysql.connector

Note: you may need to restart the kernel to use updated packages.


In [28]:
import pymysql
import pandas as pd

In [29]:
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='Sahil@123',db='department')

In [ ]:
get_response('')